In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
import json
with open('/Users/Admin/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
# import the YelpAPI Class
from yelpapi import YelpAPI

# Create an instance with your key
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [4]:
# set our API call parameters 
LOCATION = 'New Haven, CT'
TERM = 'Seafood'

In [6]:
# Specifying JSON_FILE filename
JSON_FILE = "Data/results_in_progress_CT_Seafood.json"
JSON_FILE

'Data/results_in_progress_CT_Seafood.json'

In [7]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_CT_Seafood.json not found. Saving empty list to file.


In [9]:
# our API call
search_results = yelp_api.search_query(location='New Haven, CT',
                                       term='Seafood')
print(type(search_results))
search_results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [11]:
## How many results total?
total_results = search_results['total']
total_results



553

In [12]:
## How many did we get the details for?
results_per_page = len(search_results['businesses'])
results_per_page

20

In [15]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [16]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((search_results['total']-n_results)/ results_per_page)
n_pages

28

Add this page of results to .json file

In [18]:
# join new results with old list with extend and save to file
previous_results.extend(search_results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [19]:
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/28 [00:00<?, ?it/s]

In [20]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    search_results = yelp_api.search_query(location='New Haven, CT',
                                    term='Seafood', 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(search_results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)



  0%|          | 0/28 [00:00<?, ?it/s]

In [21]:
# loading our final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,B6VlKBNeVpN4delxO2G_nA,shell-and-bones-oyster-bar-and-grill-new-haven-2,Shell & Bones Oyster Bar and Grill,https://s3-media1.fl.yelpcdn.com/bphoto/K1ic7Y...,False,https://www.yelp.com/biz/shell-and-bones-oyste...,811,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 41.280462628599174, 'longitude': ...","[pickup, delivery]",$$$,"{'address1': '100 S Water St', 'address2': '',...",+12037873466,(203) 787-3466,1994.395259
1,_MmBzKbN2Ec3lETacBofJg,fair-haven-oyster-new-haven-16,Fair Haven Oyster,https://s3-media2.fl.yelpcdn.com/bphoto/Cf4R0H...,False,https://www.yelp.com/biz/fair-haven-oyster-new...,34,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 41.31411, 'longitude': -72.88731}",[],NaN,"{'address1': '307 Front St', 'address2': '', '...",+12037455741,(203) 745-5741,3941.063207
2,BeLg5ZI2x34yqNtwiUhw_Q,stowes-seafood-west-haven,Stowe's Seafood,https://s3-media3.fl.yelpcdn.com/bphoto/ubNzhI...,False,https://www.yelp.com/biz/stowes-seafood-west-h...,399,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 41.258886, 'longitude': -72.941949}",[delivery],$$,"{'address1': '347 Beach St', 'address2': '', '...",+12039341991,(203) 934-1991,4517.014908
3,2xX2FdvndEFoz4-8eGMU8w,d-amatos-seafood-new-haven-2,D'Amato's Seafood,https://s3-media1.fl.yelpcdn.com/bphoto/_Ht-nV...,False,https://www.yelp.com/biz/d-amatos-seafood-new-...,74,"[{'alias': 'seafoodmarkets', 'title': 'Seafood...",4.5,"{'latitude': 41.3185252319592, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '423 Whalley Ave', 'address2': No...",+12034982722,(203) 498-2722,2697.566514
4,BNHXedSBgOWRtK9rImHq1A,sandbar-seafood-west-haven,Sandbar Seafood,https://s3-media4.fl.yelpcdn.com/bphoto/R0bGKY...,False,https://www.yelp.com/biz/sandbar-seafood-west-...,119,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 41.280634, 'longitude': -72.9395771}","[pickup, delivery]",$$,"{'address1': '31 Kimberly Ave', 'address2': ''...",+12039333700,(203) 933-3700,2200.704509


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
548,-NTCrJ9ouItMtPcf2obcXw,bella-napoli-pizza-milford,Bella Napoli Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/vWQzHB...,False,https://www.yelp.com/biz/bella-napoli-pizza-mi...,97,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.5,"{'latitude': 41.2322836, 'longitude': -73.0526...",[delivery],$$,"{'address1': '864 Boston Post Rd', 'address2':...",+12038771102,(203) 877-1102,12634.556996
549,7FYsi_ZZMtHsrjjzVKrAVQ,silver-sands-pizza-milford,Silver Sands Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/n3DSe8...,False,https://www.yelp.com/biz/silver-sands-pizza-mi...,54,"[{'alias': 'pizza', 'title': 'Pizza'}]",3.0,"{'latitude': 41.22669, 'longitude': -73.06991}","[pickup, delivery]",$$,"{'address1': '415 Boston Post Rd', 'address2':...",+12038820220,(203) 882-0220,14194.208584
550,9PIqSAnGl6MmuKmg_cTp3Q,star-pizza-stratford-2,Star Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/0HArXZ...,False,https://www.yelp.com/biz/star-pizza-stratford-...,31,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.5,"{'latitude': 41.19834, 'longitude': -73.13373}","[pickup, delivery]",$,"{'address1': '1101 Barnum Ave', 'address2': ''...",+12033869243,(203) 386-9243,20350.312910
551,o8vOaYYCxK5JnxiBD1eRJA,wayback-burgers-stratford,Wayback Burgers,https://s3-media2.fl.yelpcdn.com/bphoto/mMHkT6...,False,https://www.yelp.com/biz/wayback-burgers-strat...,75,"[{'alias': 'tradamerican', 'title': 'American ...",3.0,"{'latitude': 41.19924, 'longitude': -73.13146}","[pickup, delivery]",$$,"{'address1': '2850 Main St', 'address2': '', '...",+12039083400,(203) 908-3400,20147.693349
552,l_uy4MegoZJS5_zDfs4kDw,hong-kong-tokyo-stratford,Hong Kong Tokyo,https://s3-media3.fl.yelpcdn.com/bphoto/o_1azp...,False,https://www.yelp.com/biz/hong-kong-tokyo-strat...,55,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",3.0,"{'latitude': 41.19718519320253, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '411 Barnum Ave', 'address2': '',...",+12033789211,(203) 378-9211,19688.492862


In [22]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

In [23]:
# save the final results to a compressed csv
final_df.to_csv('Data/results_in_progress_CT_Seafood.csv.gz', compression='gzip',index=False)